In [10]:
import numpy as np
import pandas as pd
import os
import alpaca_trade_api as tradeapi
import datetime as dt
import pytz
from MCForecastTools import MCSimulation

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [13]:
## 2018 - 2020 Returns

timeframe = "1D"

# Set start and end datetimes between now and 3 years ago.
start_date = pd.Timestamp("2018-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-12-31", tz="America/New_York").isoformat()

# Set the ticker information
# UPDATE WITH COMPANY TICKERS
tickers = ["AMZN","COST","WMT"]

# Get 3 year's worth of historical price data for Ford, Tesla, GMC
three_year_returns = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

# Display sample data
three_year_returns.head()

AMZN                                      \
                              open     high      low    close   volume   
time                                                                     
2018-01-02 00:00:00-05:00  1172.00  1190.00  1170.51  1189.00  1615329   
2018-01-03 00:00:00-05:00  1188.30  1205.49  1188.30  1202.74  1979510   
2018-01-04 00:00:00-05:00  1207.52  1215.87  1204.66  1209.64  1918318   
2018-01-05 00:00:00-05:00  1217.51  1229.14  1210.00  1229.07  2134446   
2018-01-08 00:00:00-05:00  1236.00  1253.08  1232.03  1246.95  2621390   

                             COST                                        WMT  \
                             open     high      low   close   volume    open   
time                                                                           
2018-01-02 00:00:00-05:00  187.19  190.680  186.750  188.36  1746783   99.30   
2018-01-03 00:00:00-05:00  188.52  192.100  188.380  190.61  2258752   98.85   
2018-01-04 00:00:00-05:00  192.00  192.300  187.561  189.12  2566125   99.54   
2018-01-05 00:00:00-05:00  189.43  189.680  186.720  187.75  2000923   99.64   
2018-01-08 00:00:00-05:00  187.32  188.979  187.250  188.47  1126558  100.25   

                                                                
                             high      low    close     volume  
time                                                            
2018-01-02 00:00:00-05:00   99.79   98.520   98.560  8501212.0  
2018-01-03 00:00:00-05:00  100.00   98.770   99.440  7105903.0  
2018-01-04 00:00:00-05:00   99.78   98.911   99.535  5038573.0  
2018-01-05 00:00:00-05:00  100.38   99.610  100.120  5596520.0  
2018-01-08 00:00:00-05:00  101.75  100.210  101.570  6263264.0

In [14]:
# Create and empty DataFrame for closing prices
threeyear_daily_close = pd.DataFrame()

In [15]:
threeyear_daily_close["AMZN"] = three_year_returns["AMZN"]["close"]
threeyear_daily_close["COST"] = three_year_returns["COST"]["close"]
threeyear_daily_close["WMT"] = three_year_returns["WMT"]["close"]

In [16]:
# Drop the time component of the date
threeyear_daily_close.index = threeyear_daily_close.index.date

In [17]:
threeyear_dailyreturns = threeyear_daily_close.pct_change().dropna()

In [18]:
threeyear_dailyreturns.tail(10)

,AMZN,COST,WMT
2020-12-17,-0.001217,0.002301,0.004332
2020-12-18,-0.011104,-0.008966,-0.003149
2020-12-21,0.001459,-0.004524,0.002404
2020-12-22,0.000243,-0.008814,-0.012058
2020-12-23,-0.006172,-0.001077,-0.006657
2020-12-24,-0.004886,0.007935,0.001815
2020-12-28,0.035571,0.017912,0.011986
2020-12-29,0.011191,0.004689,-0.006266
2020-12-30,-0.010498,0.004425,-0.000485
2020-12-31,-0.008960,0.006115,-0.001040


In [23]:
cum_returns = (1 + threeyear_dailyreturns).cumprod() - 1

In [24]:
cum_returns.tail()

,AMZN,COST,WMT
2020-12-24,1.667082,0.935496,0.455966
2020-12-28,1.761951,0.970164,0.473417
2020-12-29,1.792860,0.979401,0.464184
2020-12-30,1.763541,0.988161,0.463474
2020-12-31,1.738780,1.000319,0.461952


Cumulative Returns from 2018

AMZN: 173.878%
COST: 100.032%
WMT: 46.1952%